In [1]:
import os
import glob
import pandas as pd
import numpy as np

W pierwszej kolejności należy załadować:

- plik z nazwami plików do przetworzenia (gdzie nazwy plikow są tożsame z nazwami stacji),
- ścieżkę do folderu z plikami do przetworzenia.


In [2]:
points_names_list = pd.read_csv(r'C:\Users\dell\Documents\GitHub\CTD-data-preparation\CAMPAIGN\station_names.txt',
                                sep=" ",
                                dtype = 'str',
                                encoding='UTF-8')

Jedynej kolumnie w DataFrame z nazwami stacji nadaję nagłówek, który potem przyda się do dodawania kolumn. 
Z folderu hurtowo wczytuję pliki .txt do listy. W ten sposób stworzyłam genrator będący "półproduktem", który sprzyjaja małemu zurzyciu pamięci RAM. 

In [3]:
points_names_list.columns = ['station']

In [4]:
path = r'C:\Users\dell\Documents\Loc_repo\How_to_prepare_data\CAMPAIGN' 
all_files_paths = glob.glob(os.path.join(path, "*.000"))     


df_from_each_file = list(pd.read_csv(file, sep = "\t+", header = None, engine = 'python', 
                                     names = [1,2,3,4,5,6,7,8]) for file in all_files_paths)

Do kazdego DF z listy dodaje kolumnę z odpowiednią nazwą stacji, by istniała możliwość ropoznania plików. W obecnym stanie pliki zawierają zbędne nagłówki, których należy się pozbyć w nastpnym kroku.

In [5]:
for file in range(len(df_from_each_file)):
    df_from_each_file[file][8] = points_names_list['station'].iloc[file]

In [6]:
df_from_each_file[0].head(15)#generator - półprodukt; robiac to [] tworzy się lista obiektów - wiecej miejsca

,1,2,3,4,5,6,7,8
0,Now: 12/07/2019 16:22:15,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
1,Battery Level: 1.2V,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
2,MiniCTD: S/N 49833,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
3,Site info: ORLEN,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
4,Calibrated: 09/11/2018,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
5,Latitude: 0.000000,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
6,Mode: M4,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
7,Tare: 10.0934,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
8,Pressure units: dBar,NaN,NaN,NaN,NaN,NaN,NaN,WA_01
9,12/07/2019 16:22:15,0.001,16.932,0.011,0.012,-1.202,1472.557,WA_01


In [7]:
print(len(df_from_each_file)) #załdowane zostały 44 pliki 

43


In [8]:
dfs_with_stations = []

for df in range(len(df_from_each_file)):
    dfs_with_stations.append(df_from_each_file[df].drop(df_from_each_file[df].index[:9]))

Usunięcie nagłówków powoduje usunięcie części indeksów - by wykonać iterację, indeksy muszą być w odpowiedniej kolejności. Dlatego kolejną operacją jest reset indeksów (wykonuję to po każdej ingerencji w indeksy).


In [9]:
dfs_with_reset_index = []

for df in range(len(dfs_with_stations)):
    dfs_with_reset_index.append(dfs_with_stations[df].reset_index(drop=True))

In [10]:
dfs_with_reset_index[0].head(10)

,1,2,3,4,5,6,7,8
0,12/07/2019 16:22:15,0.001,16.932,0.011,0.012,-1.202,1472.557,WA_01
1,12/07/2019 16:22:15,0.001,16.934,0.012,0.013,-1.202,1472.565,WA_01
2,12/07/2019 16:22:15,0.002,16.935,0.009,0.012,-1.204,1472.566,WA_01
3,12/07/2019 16:22:15,0.003,16.935,0.010,0.012,-1.203,1472.567,WA_01
4,12/07/2019 16:22:16,0.001,16.936,0.012,0.013,-1.203,1472.571,WA_01
5,12/07/2019 16:22:16,0.001,16.935,0.012,0.013,-1.203,1472.568,WA_01
6,12/07/2019 16:22:16,0.002,16.936,0.012,0.013,-1.203,1472.571,WA_01
7,12/07/2019 16:22:16,0.002,16.936,0.012,0.013,-1.203,1472.571,WA_01
8,12/07/2019 16:22:17,0.002,16.936,0.011,0.012,-1.203,1472.571,WA_01
9,12/07/2019 16:22:17,0.002,16.937,0.011,0.012,-1.203,1472.574,WA_01


Przechodzę do czyszczenia danych z błędnych rejestracji. 
Usuwanie danych należy wykonać dla każdego załadowanego pliku z listy osobno, gdyż nie tworzą one serii danych ciągłych.

Iterację wykonuję na podstawie kolumny zawierającej wartości ciśnienia. Kolejne komórki w kolumnie należy porównać tak, aby  poprawne dane zawierały rosnący ciąg liczb. Po wykonaniu iteracji, resetuję indeksy w nowej liście z DF.

In [11]:
index_to_erase = []   
data_with_corr_pressure = []

for df in range(len(dfs_with_reset_index)):
    current_lowest_value = dfs_with_reset_index[df].iloc[0, 1]

    for x in range(0, len(dfs_with_reset_index[df][2]) - 1):

        if current_lowest_value >= dfs_with_reset_index[df].iloc[x + 1, 1]:
            index_to_erase.append(x + 1)
        else:
            current_lowest_value = dfs_with_reset_index[df].iloc[x + 1, 1]
        
    data_with_corr_pressure.append(dfs_with_reset_index[df].drop(index_to_erase))
    index_to_erase = []   

In [12]:
data_with_corr_pressure_fixed_index = []

for df in range(len(data_with_corr_pressure)):
    data_with_corr_pressure_fixed_index.append(data_with_corr_pressure[df].reset_index(drop=True))

In [13]:
data_with_corr_pressure_fixed_index[0].head()

,1,2,3,4,5,6,7,8
0,12/07/2019 16:22:15,0.001,16.932,0.011,0.012,-1.202,1472.557,WA_01
1,12/07/2019 16:22:15,0.002,16.935,0.009,0.012,-1.204,1472.566,WA_01
2,12/07/2019 16:22:15,0.003,16.935,0.010,0.012,-1.203,1472.567,WA_01
3,12/07/2019 16:22:37,0.243,16.506,9.467,6.460,3.811,1478.660,WA_01
4,12/07/2019 16:22:37,0.400,16.603,10.879,7.486,4.577,1480.164,WA_01


Otrzymane dane wciąż nie są idealne - wśród nich występują błędne zapisy świadące o tym, że rejestracja rozpoczęłą się przed zanurzeniem sondy do wody. Tych zapisów pozbędę się poprzez iteracji w oparciu o kolumnę z gęstością [5] ustalając, że akceptowalne wartości to te, które zostały zarejestrowane, gdy gęstość wody wyniosła 3,5 g/m3 lub więcej. 

In [16]:
index_to_erase_density = []
corrected_data = []
 
for df in range(len(data_with_corr_pressure_fixed_index)):   
    current_dens_value = data_with_corr_pressure_fixed_index[df].iloc[0, 5] 
    
    for x in range(0, len(data_with_corr_pressure_fixed_index[df][5])-1): 
        
            if current_dens_value <= 3.5: 
                index_to_erase_density.append(x)
                current_dens_value = data_with_corr_pressure_fixed_index[df].iloc[x+1, 5]
                
            else:
                current_dens_value = data_with_corr_pressure_fixed_index[df].iloc[x+1, 5]

    corrected_data.append(data_with_corr_pressure_fixed_index[df].drop(index_to_erase_density))
    index_to_erase_density = []

In [17]:
corrected_data_index = []

for df in range(len(corrected_data)):
    corrected_data_index.append(corrected_data[df].reset_index(drop=True))

In [18]:
corrected_data_index[0].head(15)

,1,2,3,4,5,6,7,8
0,12/07/2019 16:22:37,0.243,16.506,9.467,6.460,3.811,1478.660,WA_01
1,12/07/2019 16:22:37,0.400,16.603,10.879,7.486,4.577,1480.164,WA_01
2,12/07/2019 16:22:37,0.528,16.666,10.849,7.452,4.540,1480.334,WA_01
3,12/07/2019 16:22:38,0.662,16.704,10.859,7.452,4.534,1480.461,WA_01
4,12/07/2019 16:22:38,0.739,16.723,10.840,7.435,4.518,1480.505,WA_01
5,12/07/2019 16:22:38,0.764,16.732,10.847,7.438,4.519,1480.539,WA_01
6,12/07/2019 16:22:38,0.819,16.735,10.872,7.456,4.532,1480.570,WA_01
7,12/07/2019 16:22:39,0.876,16.731,10.872,7.457,4.534,1480.559,WA_01
8,12/07/2019 16:22:39,0.937,16.723,10.874,7.460,4.538,1480.537,WA_01
9,12/07/2019 16:22:39,1.040,16.726,10.851,7.442,4.524,1480.528,WA_01


Po wyczyszczeniu danych z błędnych zapisów łączę poszczególne pliki do jednej ramki danych metodą .concat() oraz nadaję finalne nagłówki.  

In [19]:
concatenated_data = pd.concat(corrected_data_index, axis=0, ignore_index=False, sort=False)
concatenated_data.columns = ["Date", "Pressure", "Temperature", "Conductivity","Salinity", "Density", "SOS", "Station"]

In [20]:
concatenated_data.shape

(17731, 8)

In [21]:
concatenated_data.head()

,Date,Pressure,Temperature,Conductivity,Salinity,Density,SOS,Station
0,12/07/2019 16:22:37,0.243,16.506,9.467,6.460,3.811,1478.660,WA_01
1,12/07/2019 16:22:37,0.400,16.603,10.879,7.486,4.577,1480.164,WA_01
2,12/07/2019 16:22:37,0.528,16.666,10.849,7.452,4.540,1480.334,WA_01
3,12/07/2019 16:22:38,0.662,16.704,10.859,7.452,4.534,1480.461,WA_01
4,12/07/2019 16:22:38,0.739,16.723,10.840,7.435,4.518,1480.505,WA_01


In [22]:
concatenated_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17731 entries, 0 to 266
Data columns (total 8 columns):
Date            17731 non-null object
Pressure        17731 non-null float64
Temperature     17731 non-null float64
Conductivity    17703 non-null float64
Salinity        17703 non-null float64
Density         17703 non-null float64
SOS             17703 non-null float64
Station         17731 non-null object
dtypes: float64(6), object(2)
memory usage: 1.2+ MB


In [23]:
concatenated_data.describe()

,Pressure,Temperature,Conductivity,Salinity,Density,SOS
count,17731.000000,17731.000000,17703.000000,17703.000000,17703.000000,17703.000000
mean,19.677970,14.716821,10.464672,7.517940,4.980143,1473.953546
std,11.448178,2.559746,0.608453,0.178212,0.433779,9.015025
min,0.007000,0.011000,8.351000,6.136000,3.502000,1444.266000
25%,9.972500,15.254000,10.535000,7.490000,4.819000,1475.965500
50%,19.443000,15.466000,10.666000,7.550000,4.897000,1476.785000
75%,29.292500,15.801000,10.808000,7.592000,5.054000,1477.847500
max,44.241000,20.346000,13.222000,8.909000,6.227000,1491.639000


In [1]:
concatenated_data.to_csv("corrected_data")

NameError: name 'concatenated_data' is not defined

In [ ]:
#cleaned_data = concatenated_df.replace(np.NaN, " ", regex=True)